In [ ]:
import os
import sys
import logging

import numpy as np
import pandas as pd
from sagemaker.local import LocalSession
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.sklearn.processing import SKLearnProcessor

In [ ]:
session = LocalSession()
session.config = {"local": {"local_code": True}}
bucket = session.default_bucket()
role = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001"
region = "eu-central-1"
raw_input_path = f"s3://{bucket}/imdb/processing/raw/small/raw.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

## Data pre-processing

To run the scikit-learn preprocessing script as a processing job, create a `SKLearnProcessor`, which lets you run scripts inside of processing jobs using the scikit-learn image provided.

In [ ]:
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="local", instance_count=1
)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="source/preprocessing.py",
    inputs=[ProcessingInput(source=raw_input_path, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test":
        preprocessed_test_data = output["S3Output"]["S3Uri"]

## Training

In [ ]:
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased",
}
estimator = PyTorch(
    entry_point="train.py",
    source_dir="source",
    role=role,
    framework_version="1.7.1",
    py_version="py3",
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
)
estimator.fit({'train': preprocessed_training_data, 'test': preprocessed_test_data})